In [336]:
import re
from collections import Counter
import overpy 
from random import randint
from training.xml_writer import AddressesXML
from address_templeter import LABELS, STREET_PRETEXT, \
    PLACE_PRETEXT, REGION_PRETEXT, HOUSE_PRETEXT,\
    PUNCTUATION_MARK, DASH, tokenize, in_pre
api = overpy.Overpass()

In [348]:
geo_polygon = "(50.462287,30.499273,50.530674,30.564974)"
response = api.query(f"""way["addr:street"]["addr:housenumber"]{geo_polygon};
    (._;>;);
    out body;""")

In [349]:
address_way = [way.tags for way in response.ways]


In [350]:
count = Counter()
for adr in address_way:
    for word in re.findall(r"\w+", adr.get('name', '')):
        if word[0].islower():
            count[word]+=1
[s for s in count if len(s)> 3]

['виховний',
 'комплекс',
 'ринок',
 'центр',
 'маркет',
 'юнацька',
 'футбольна',
 'школа',
 'дитяча',
 'поліклініка',
 'району',
 'районна',
 'державна',
 'адміністрація',
 'motors',
 'обслуговування',
 'церква',
 'тітка',
 'районний',
 'фінансів',
 'університету',
 'академія',
 'слідче',
 'управління',
 'податкова',
 'інспекція',
 'офис',
 'соціального',
 'захисту',
 'інвалідів',
 'надання',
 'адміністративних',
 'послуг',
 'метро',
 'титул',
 'здорового',
 'материнства',
 'національний',
 'університет',
 'театру',
 'кіно',
 'телебачення',
 'імені',
 'моделей',
 'обуви',
 'склады',
 'сьомого',
 'корпус',
 'гідробіології',
 'археології',
 'мистецтв',
 'радіовежа',
 'української',
 'культури',
 'мистецтва',
 'морського',
 'річкового',
 'флоту',
 'садок',
 'дитячий',
 'ластівка',
 'ліцею',
 'стоматологическая',
 'поликлиника',
 'оздоровчо',
 'екологічний',
 'інтернат',
 'будинок',
 'тубінфікованих',
 'славістичний',
 'поліції',
 'охорони',
 'початкова',
 'відокремлений',
 'підрозділ',


In [339]:
iter_address = iter(address_way)

In [340]:
def addr_housenumber(housenumber):
    
    def mark_house(part_house):
        if part_house in PUNCTUATION_MARK:
            return "PunctuationMark", part_house
        elif part_house == '-':
            return "Dash", part_house
        else:
            return 'HouseNumber', part_house
    result = []
    housenumber = re.findall("\d+|\w+|[/]", housenumber)
    variant = randint(1,4)
    if len(housenumber) > 3:
        variant = randint(1,2)
    patterns = {1:["".join(housenumber)], 2:housenumber, 3:["-".join(housenumber)], 
                4:[housenumber[0], "-", housenumber[1] if len(housenumber) >=2 else "".join(housenumber)]}
    for s in patterns.get(variant):
        result.append(mark_house(s))
    return result

def addr_name(name):
    result = []
    splited = re.findall(r"[\w]+|\,\(\)\!\?", name)
    for s in splited:
        if s in PUNCTUATION_MARK:
            result.append(("PunctuationMark", s))
        else:
            result.append(("HouseName", s))
    return result

def post_code(code):
    return [("PostCode", code)]

def street(street_name):
    result =[]
    splited = re.findall(r"[\w]+|\,\(\)\!\?", street_name)
    for s in splited:
        if s in PUNCTUATION_MARK:
            result.append(("PunctuationMark", s))
        if s in STREET_PRETEXT:
            result.append(("StreetPretext", s)) 
        else:
            result.append(("Street", s))
    if randint(0,1):
        pre = result.pop(-1)
        result.insert(0, pre)
    return result

def other(text):
    result = []
    splited = re.findall(r"[\w]+|\,\(\)\!\?", text)
    for s in splited:
        if s in PUNCTUATION_MARK:
            result.append(("PunctuationMark", s))
        if s.isdigit():
            result.append(("OtherNumber", s)) 
        else:
            result.append(("OtherText", s))
    return result

In [341]:
def get_row():
    rnd = randint(0,3)
    row = [[("PlacePretext", "г"), ("Place", "Киев"),], 
           [("PlacePretext", "г"), ("Place", "Киев"),("Dash", "-"), ("Place", "Подольск"),], 
           [("PlacePretext", "г"), ("Place", "Киев-Подольск")], 
           [("PlacePretext", "г"), ("Place", '"Киев"')]]
    return row[rnd]

func_dict = {'addr:housenumber':addr_housenumber, 
             'addr:postcode': post_code, 
             'addr:street':street,
             'name': addr_name,}


In [342]:
def pars_address(address):
    row = get_row()
    address = {'addr:street':address.get('addr:street'), 
               'addr:housenumber': address.get('addr:housenumber'), 
               'name':address.get('name'), 
               'addr:postcode':address.get('addr:postcode'), **address}
    address['building']=0
    address['building:levels']=0
    
    for key, value in address.items():
        if value:
            if randint(0,1):
                row.append(("PunctuationMark", ","))
            for v in func_dict.get(key, other)(value):
                
                row.append(v)
    return row

In [343]:
xml = AddressesXML()
for addr in iter_address:
    xml.append(pars_address(addr))

In [344]:
xml.write("/home/maksim/PycharmProjects/adressparser/training/checked_osm.txt")

